# Advertisement Campaign Analysis

In [2]:
# packages
import pandas as pd
import os
from dotenv import load_dotenv
from google.cloud import bigquery

### Step 1: Data Wrangling

Within our query, we perform the following preprocessing steps:
1. Restructure our data by unioning two tables
2. Clean up the date data type
3. Handle nulls with interpolation
4. Enrich the dataset with calculations

In [ ]:
# retreive data from bigquery

load_dotenv()
GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

# Initialize the BigQuery Client
client = bigquery.Client()

# BQ query
sql_query = """
        WITH 
            campaign_union AS (
                -- combine our control and test data sets
                SELECT 
                    * 
                FROM `crudek-data.practice_data.campaign_cntrl`
                UNION ALL
                SELECT 
                    * 
                FROM`crudek-data.practice_data.campaign_test`
            ),
            campaign_clean AS (
                SELECT 
                    campaign_name,
                    -- format as a date
                    PARSE_DATE('%d.%m.%Y', date) AS date,
                    spend_usd,
                    -- handle missing data with linear interpolation (could also remove, impute, etc.)
                    COALESCE(impressions, (LAG(impressions) OVER (ORDER BY campaign_name, date) + LEAD(impressions) OVER (ORDER BY campaign_name, date)) / 2) AS impressions,
                    COALESCE(reach, (LAG(reach) OVER (ORDER BY campaign_name, date) + LEAD(reach) OVER (ORDER BY campaign_name, date)) / 2) AS reach,
                    COALESCE(website_clicks, (LAG(website_clicks) OVER (ORDER BY campaign_name, date) + LEAD(website_clicks) OVER (ORDER BY campaign_name, date)) / 2) AS website_clicks,
                    COALESCE(searches, (LAG(searches) OVER (ORDER BY campaign_name, date) + LEAD(searches) OVER (ORDER BY campaign_name, date)) / 2) AS searches,
                    COALESCE(content_views, (LAG(content_views) OVER (ORDER BY campaign_name, date) + LEAD(content_views) OVER (ORDER BY campaign_name, date)) / 2) AS content_views,
                    COALESCE(add_to_cart, (LAG(add_to_cart) OVER (ORDER BY campaign_name, date) + LEAD(add_to_cart) OVER (ORDER BY campaign_name, date)) / 2) AS add_to_cart,
                    COALESCE(purchases, (LAG(purchases) OVER (ORDER BY campaign_name, date) + LEAD(purchases) OVER (ORDER BY campaign_name, date)) / 2) AS purchases
                FROM campaign_union
                ORDER BY campaign_name, date
            ),
            campaign_calcs AS (
                SELECT
                    campaign_name,
                    date,
                    (purchases / website_clicks) AS conversion_rate,
                    (website_clicks / impressions) AS click_through_rate, -- CTR
                    (spend_usd / website_clicks) AS cost_per_click, -- CPC
                    (spend_usd / purchases) AS customer_acq_cost -- CAC   
                FROM campaign_clean
            )
        SELECT *
        FROM campaign_calcs
        ORDER BY campaign_name, date
"""

query_job = client.query(
    sql_query)

# retreive and convert the result to a Pandas DataFrame
df = query_job.to_dataframe()

In [10]:
print(df.head())
print(df.tail())

      campaign_name        date  conversion_rate  click_through_rate  \
0  Control Campaign  2019-08-01         0.088084            0.084835   
1  Control Campaign  2019-08-02         0.063009            0.067003   
2  Control Campaign  2019-08-03         0.057160            0.049411   
3  Control Campaign  2019-08-04         0.110930            0.042057   
4  Control Campaign  2019-08-05         0.155646            0.038982   

   cost_per_click  customer_acq_cost  
0        0.324971           3.689320  
1        0.216646           3.438356  
2        0.360018           6.298387  
3        0.632953           5.705882  
4        0.517412           3.324275  
    campaign_name        date  conversion_rate  click_through_rate  \
55  Test Campaign  2019-08-26         0.074346            0.047253   
56  Test Campaign  2019-08-27         0.119814            0.057729   
57  Test Campaign  2019-08-28         0.088531            0.149084   
58  Test Campaign  2019-08-29         0.088485       